## CNN with Tensorflow

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline

c:\users\hp\anaconda2\envs\translator\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#import data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
#Load datasets
train=mnist.train
val=mnist.validation
test=mnist.test

## Implementation of a 2-layer CNN with TensorFlow

In [4]:
units=200
lr=0.01
iterations_n=30000
batch_size=200

## Train

30,000 iterations

batch size is 250

learning rate is 0.01

kernel size is 3

neurons in hidden unites are 200.

86 filters in each conv layer.

Max pooling with kernel size of 2
    
Keep_prob=0.75 for dropout

In [7]:
#Inputs
X= tf.placeholder(tf.float32, [None, 784])
x=tf.reshape(X, [-1, 28,28,1])
Y=tf.placeholder(tf.int32, [None, 10])

#First conv
with tf.name_scope('conv1') as scope:
    kernel = tf.Variable(tf.truncated_normal([3, 3, 1, 86], dtype=tf.float32, stddev=1e-1), name='weights')
    bias=tf.Variable(tf.zeros([86]))
    conv_1 = tf.nn.conv2d(x, kernel, [1, 1, 1, 1], padding='SAME')+bias
    conv_1= tf.nn.relu(conv_1, name=scope)

#Pooling
pool1 = tf.nn.max_pool(conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')    
    
#second conv
with tf.name_scope('conv2') as scope:
    kernel = tf.Variable(tf.truncated_normal([3, 3, 86, 86], dtype=tf.float32, stddev=1e-1), name='weights')
    bias=tf.Variable(tf.zeros([86]))
    conv_2 = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
    conv_2= tf.nn.relu(conv_2, name=scope)
#pooling    
pool2 = tf.nn.max_pool(conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')     
#fully conected 
with tf.name_scope('ml1') as scope:
    #layer1=tf.reshape(conv_2, [-1, 28*28*32])
    layer1=tf.contrib.layers.flatten(pool2)
    W=tf.Variable(tf.truncated_normal([int(layer1.get_shape()[1]), units]))
    b=tf.Variable(tf.zeros([units]))
    mul=tf.matmul(layer1, W)+b
    ml_1=tf.sigmoid(mul)

    
dropout=tf.nn.dropout(ml_1, 0.75)
#fully conected
with tf.name_scope('ml2') as scope:
    W=tf.Variable(tf.truncated_normal([units, 10]))
    b=tf.Variable(tf.zeros([10]))
    y_hat=tf.matmul(dropout, W)+b

#Loss function
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_hat))
#Optimizer
optimizer=tf.train.GradientDescentOptimizer(lr)
training_op=optimizer.minimize(cross_entropy)
# Evaluation
correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:

valid_accs=[0]
losses=[]
stop=0
with tf.Session() as sess:
    init=tf.global_variables_initializer()
    init.run()
    for iters in range(iterations_n):
        if stop>3:
            print ('EARLY STOP, EPOCH: ', iters)
            break
        if iters % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(batch_size)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={X: batch[0], Y: batch[1]})
            #print('step %d, validation accuracy %g' % (iters, validation_accuracy))

            if validation_accuracy<valid_accs[-1]:
                stop+=1
            else:
                stop=0
            valid_accs.append(validation_accuracy)
        X_batch, y_batch=train.next_batch(batch_size)
        loss,_=sess.run([cross_entropy, training_op], feed_dict={X: X_batch, Y: y_batch})
        losses.append(loss)
    test_accuracy=accuracy.eval(feed_dict={X: test.images, Y: test.labels})
    val_accuracy=accuracy.eval(feed_dict={X: val.images, Y: val.labels})
print ('Iterations:', iterations_n)
print('Validation accuracy: ', val_accuracy)
print ('Test accuracy: ',  test_accuracy)